In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.neural_network import MLPRegressor

In [20]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/var/folders/nj/rlswb9ks3qzd3ch3nj0cg2qh0000gn/T/ipykernel_7649/3462665596.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [21]:
df_train = pd.read_csv('./Dataset/train.csv', index_col='Id')
df_train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [22]:
targets_train = df_train.SalePrice
df_train.drop(columns=['SalePrice'], axis=1, inplace=True)
print('Length of the targets : ', len(targets_train))
targets_train.head()

Length of the targets :  1460


Id
1    208500
2    181500
3    223500
4    140000
5    250000
Name: SalePrice, dtype: int64

In [23]:
targets_train_log = np.log(targets_train)

In [24]:
df_train_num = df_train.select_dtypes(exclude='object')
df_train_cat = df_train.select_dtypes(include='object')

print('Shape of the dataset with numerical columns : ', df_train_num.shape)
print('Shape of the dataset with categorical columns : ', df_train_cat.shape)

Shape of the dataset with numerical columns :  (1460, 36)
Shape of the dataset with categorical columns :  (1460, 43)


In [25]:
num_cols = df_train_num.columns
print('Number of Numerical columns : ', len(num_cols))
num_cols

Number of Numerical columns :  36


Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

In [26]:
cat_cols = df_train_cat.columns
print('Number of Categorical columns : ', len(cat_cols))
cat_cols

Number of Categorical columns :  43


Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [27]:
discrete_num_cols = ['MSSubClass', 'OverallQual', 'OverallCond',
                     'YearBuilt', 'YearRemodAdd',  'BsmtFullBath',
                     'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
                     'GarageYrBlt', 'GarageCars', 'MoSold', 'YrSold']

cont_num_cols = list(set(num_cols) - set(discrete_num_cols))

print('Number of discrete numerical columns : ', len(discrete_num_cols))
print('Number of continuous numerical columns : ', len(cont_num_cols))


Number of discrete numerical columns :  15
Number of continuous numerical columns :  21


In [28]:
log_num_cols = ['LowQualFinSF', '3SsnPorch', 'LotArea', 'PoolArea', 'MiscVal']
ss_num_cols = list(set(cont_num_cols) - set(log_num_cols))
print('Number of numerical skewed columns : ', len(log_num_cols))
print('Number of numerical non-skewed columns : ', len(ss_num_cols))

Number of numerical skewed columns :  5
Number of numerical non-skewed columns :  16


In [29]:
impute = SimpleImputer(strategy='median')
df_train[cont_num_cols] = impute.fit_transform(df_train[cont_num_cols])
df_train[cont_num_cols].isnull().sum()

BsmtFinSF1       0
BsmtHalfBath     0
GrLivArea        0
1stFlrSF         0
BsmtUnfSF        0
MiscVal          0
3SsnPorch        0
KitchenAbvGr     0
2ndFlrSF         0
LotArea          0
OpenPorchSF      0
BsmtFinSF2       0
EnclosedPorch    0
LowQualFinSF     0
GarageArea       0
PoolArea         0
WoodDeckSF       0
ScreenPorch      0
TotalBsmtSF      0
MasVnrArea       0
LotFrontage      0
dtype: int64

In [43]:
impute_dnc = SimpleImputer(strategy='most_frequent')
df_train[discrete_num_cols] = impute_dnc.fit_transform(df_train[discrete_num_cols])
df_train[discrete_num_cols].isnull().sum()


MSSubClass      0
OverallQual     0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
BsmtFullBath    0
FullBath        0
HalfBath        0
BedroomAbvGr    0
TotRmsAbvGrd    0
Fireplaces      0
GarageYrBlt     0
GarageCars      0
MoSold          0
YrSold          0
dtype: int64

In [30]:
df_train[log_num_cols] = np.log1p(df_train[log_num_cols])
ss = StandardScaler()
df_train[cont_num_cols] = ss.fit_transform(df_train[cont_num_cols])
df_train[cont_num_cols].head()

,BsmtFinSF1,BsmtHalfBath,GrLivArea,1stFlrSF,BsmtUnfSF,MiscVal,3SsnPorch,KitchenAbvGr,2ndFlrSF,LotArea,OpenPorchSF,BsmtFinSF2,EnclosedPorch,LowQualFinSF,GarageArea,PoolArea,WoodDeckSF,ScreenPorch,TotalBsmtSF,MasVnrArea,LotFrontage
Id,,,,,,,,,,,,,,,,,,,,,
1,0.575425,-0.241061,0.370333,-0.793434,-0.944591,-0.190481,-0.128521,-0.211454,1.161852,-0.133270,0.216503,-0.288653,-0.359325,-0.133602,0.351000,-0.069393,-0.752176,-0.270208,-0.459303,0.514104,-0.220875
2,1.171992,3.948809,-0.482512,0.257140,-0.641228,-0.190481,-0.128521,-0.211454,-0.795163,0.113413,-0.704483,-0.288653,-0.359325,-0.133602,-0.060731,-0.069393,1.626195,-0.270208,0.466465,-0.570750,0.460320
3,0.092907,-0.241061,0.515013,-0.627826,-0.301643,-0.190481,-0.128521,-0.211454,1.189351,0.420049,-0.070361,-0.288653,-0.359325,-0.133602,0.631726,-0.069393,-0.752176,-0.270208,-0.313369,0.325915,-0.084636
4,-0.499274,-0.241061,0.383659,-0.521734,-0.061670,-0.190481,-0.128521,-0.211454,0.937276,0.103317,-0.176048,-0.288653,4.092524,-0.133602,0.790804,-0.069393,-0.752176,-0.270208,-0.687324,-0.570750,-0.447940
5,0.463568,-0.241061,1.299326,-0.045611,-0.174865,-0.190481,-0.128521,-0.211454,1.617877,0.878431,0.563760,-0.288653,-0.359325,-0.133602,1.698485,-0.069393,0.780197,-0.270208,0.199680,1.366489,0.641972


In [39]:
mi_scores = mutual_info_regression(df_train[cont_num_cols], targets_train_log)
MI = pd.DataFrame({'Columns' : cont_num_cols, 'MI_scores' : mi_scores}, 
                columns = ['Columns', 'MI_scores'])
MI.sort_values(ascending = False, by = 'MI_scores')

,Columns,MI_scores
2,GrLivArea,0.455642
14,GarageArea,0.364173
18,TotalBsmtSF,0.359554
3,1stFlrSF,0.297855
20,LotFrontage,0.219696
9,LotArea,0.201365
8,2ndFlrSF,0.197647
10,OpenPorchSF,0.154141
0,BsmtFinSF1,0.142070
4,BsmtUnfSF,0.125805


In [53]:
temp = MI[MI['MI_scores'] > MI['MI_scores'].mean()]
print(temp['Columns'].values)
temp

['BsmtFinSF1' 'GrLivArea' '1stFlrSF' '2ndFlrSF' 'LotArea' 'OpenPorchSF'
 'GarageArea' 'TotalBsmtSF' 'LotFrontage']


,Columns,MI_scores
0,BsmtFinSF1,0.142070
2,GrLivArea,0.455642
3,1stFlrSF,0.297855
8,2ndFlrSF,0.197647
9,LotArea,0.201365
10,OpenPorchSF,0.154141
14,GarageArea,0.364173
18,TotalBsmtSF,0.359554
20,LotFrontage,0.219696


In [41]:
df_train[discrete_num_cols].head()

,MSSubClass,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,MoSold,YrSold
Id,,,,,,,,,,,,,,,
1,60,7,5,2003,2003,1,2,1,3,8,0,2003.0,2,2,2008
2,20,6,8,1976,1976,0,2,0,3,6,1,1976.0,2,5,2007
3,60,7,5,2001,2002,1,2,1,3,6,1,2001.0,2,9,2008
4,70,7,5,1915,1970,1,1,0,3,7,1,1998.0,3,2,2006
5,60,8,5,2000,2000,1,2,1,4,9,1,2000.0,3,12,2008


In [45]:
mi_scores_dnc = mutual_info_regression(df_train[discrete_num_cols], targets_train_log)
MI_dnc = pd.DataFrame({'Columns': discrete_num_cols, 'MI_scores': mi_scores_dnc},
                  columns=['Columns', 'MI_scores'])
MI_dnc.sort_values(ascending=False, by='MI_scores')


,Columns,MI_scores
1,OverallQual,0.566360
12,GarageCars,0.370182
3,YearBuilt,0.363612
0,MSSubClass,0.271628
6,FullBath,0.264662
4,YearRemodAdd,0.242801
11,GarageYrBlt,0.235448
9,TotRmsAbvGrd,0.219512
10,Fireplaces,0.177760
2,OverallCond,0.113858


In [54]:
temp = MI_dnc[MI_dnc['MI_scores'] > MI_dnc['MI_scores'].mean()]
print(temp['Columns'].values)
temp


['MSSubClass' 'OverallQual' 'YearBuilt' 'YearRemodAdd' 'FullBath'
 'TotRmsAbvGrd' 'GarageYrBlt' 'GarageCars']


,Columns,MI_scores
0,MSSubClass,0.271628
1,OverallQual,0.566360
3,YearBuilt,0.363612
4,YearRemodAdd,0.242801
6,FullBath,0.264662
9,TotRmsAbvGrd,0.219512
11,GarageYrBlt,0.235448
12,GarageCars,0.370182


**Observations**
* Features turned out to be important using Mutual Information

| Features type | Features |
|---------------| ---------| 
| Continuous Numerical | ['BsmtFinSF1' 'GrLivArea' '1stFlrSF' '2ndFlrSF' 'LotArea' 'OpenPorchSF' 'GarageArea' 'TotalBsmtSF' 'LotFrontage'] |
| Discrete Numerical | ['MSSubClass' 'OverallQual' 'YearBuilt' 'YearRemodAdd' 'FullBath' 'TotRmsAbvGrd' 'GarageYrBlt' 'GarageCars'] |
